In [1]:
class Channel:
    def __init__(self, user,total_deposit):
        self.deposit = total_deposit
        self.user = user
        self.u2t = total_deposit/2
        self.t2u = total_deposit/2

In [2]:
class Transaction:
    def __init__(self, sender, receiver, amount, consent):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.consent = consent

In [3]:
class Tumbler:
    def __init__(self, balance):
        self.balance = balance
        self.channels = []
        
    def create_channel(self, user, desposit_amount):
        self.balance -= desposit_amount
        user.deposit(desposit_amount) 
        channel = Channel(user,desposit_amount*2)
        self.channels.append(channel)



In [4]:
class User:
    def __init__(self, user_id, balance):
        self.user_id = user_id
        self.balance = balance
        self.channel_balance = 0

    def deposit(self, amount):
        # Simulate depositing funds into the user's account
        self.balance -= amount
        self.channel_balance = amount
        return f"Deposited {amount} BTC into User {self.user_id}'s PCH account."
    
#     def initiate_tx(transaction):
        

    def withdraw(self, amount):
        # Simulate withdrawing funds from the user's account
        if amount > self.channel_balance:
            return "Insufficient funds."
        else:
            self.balance += amount
            self.channel_balance = 0
            return f"Withdrew {amount} BTC from User {self.user_id}'s PCH account."

### TumbleBit Implementation

In [5]:
import random


'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x, y = extended_gcd(b, a % b)
        return d, y, x - y * (a // b)

def find_multiplicative_inverse(a, m):
    d, x, y = extended_gcd(a, m)
    if d == 1:
        return (x % m + m) % m  # Ensure the result is positive and less than m
    else:
        raise ValueError("Multiplicative inverse does not exist.")


#returns True if a number is prime else False
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True




def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = find_multiplicative_inverse(e, phi)
    
#     print("KeyGen",e,n,d, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))




def encrypt(pk, plaintext):
    #Unpack the key into it's components
#     print(pk, plaintext)
    key, n = pk
#     print(key, n)
    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = (plaintext ** key) % n
    #Return the array of bytes
    return cipher




def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = (ciphertext ** key) % n
    #Return the array of bytes as a string
    return plain
    

In [6]:
def find_channel_by_userID(userID, channels):
    for channel in channels:
        if channel.user.user_id == userID:
            return channel
    return None

In [7]:
def tx_proto_print(*args, sep=' ', end='\n'):
#     debug = True
    debug = False
    if debug:
        output = sep.join(map(str, args)) + end
        print(output, end='')

In [8]:
def transction_protocol(sender, receiver, tumbler, amount):
    print("\n\n")
    print("***************************************************************")

    print("Sender ID = "+str(sender.user_id))
    print("Receiver ID = "+str(receiver.user_id))
    print("Amount:",amount)
    
    sender_channel = find_channel_by_userID(sender.user_id, tumbler.channels)
    receiver_channel = find_channel_by_userID(receiver.user_id, tumbler.channels)
    
    if amount>sender_channel.u2t:
        print("Smart Contract: TX invalid, sender doesn't have enough balance!")
        return
    
    if amount>receiver_channel.t2u:
        print("Error! ET1: Receiver channel doesn't have enough balance!")
        return
    
    print("Information about Sender and the channel between Sender and Tumbler:")
    print(vars(sender))
    print(vars(sender_channel))
    print("Information about Receiver and the channel between Tumbler and Receiver:")
    print(vars(receiver))
    print(vars(receiver_channel))
    
    tx_proto_print("\n\n")
    tx_proto_print("***************************************************************")
    tx_proto_print("             Tumble Bit Transaction Protocol                   ")
    tx_proto_print("***************************************************************")

    tx_proto_print("TMB = Tumbler (a trustless 3rd party mediator)")
    tx_proto_print("\n\n--------------- Generating (z,c) at TMB ------------------------")
    temp=random.randrange(1000, 1500)
    while(is_prime(temp) != True):
        temp = random.randrange(1, 50)
    p = temp #taking any random no
    q = 197 #taking any random no

    #Generating your public/private keypairs
    public, private = generate_keypair(p, q)
    tx_proto_print("TMB (RSA public key) = ", public ," TMB (RSA private key) = ", private)
    
    while(amount):

        epsilon = random.randrange(1, 50)
        sigma = random.randrange(1, 50)
        tx_proto_print("Sigma = ",sigma,"epsilon = ",epsilon)
        z = decrypt(public, epsilon)
        c = sigma + epsilon


        tx_proto_print("\n\n\n")
        tx_proto_print("-------------- [TMB ---> Receiver Communication]-------------------")
        tx_proto_print("TMB sending (z,c) = (",z,", ",c,") to Receiver")




        tx_proto_print("\n\n\n")
        tx_proto_print("-------------- [Receiver ---> Sender Communication]-------------------")
        tx_proto_print("Blinding the z before sending")
        bf = random.randrange(1, 10)
        tx_proto_print("Blinding factor bf = ", bf,"& send zStar = z*(bf^p.k)")

        e,n = public
        zStar = (z*decrypt(public, bf))%n
        tx_proto_print("Receiver sending zStar Z* to Sender = ",zStar)
        tx_proto_print("verify zStar=",decrypt(public, bf*epsilon))




        tx_proto_print("\n\n\n")
        tx_proto_print("--------------[Sender ---> TMB Communication]-------------------")
        tx_proto_print("Sender request solution of zStar (z*) to TMB for 1 coin")


        epsilonStar=encrypt(private, zStar)
        tx_proto_print("epsilonStar = ",epsilonStar)
        x = random.randrange(1, 500)
        y = hash(x)
        q = epsilonStar + x



        tx_proto_print("Payment solver protocol exchange")
        for x in range(1,5):
            bff = random.randrange(1000, 5000)
            tx_proto_print("Blinding factors bf = ", bf)
        for x in range(1,5):
            temp = random.randrange(1000, 5000)
            tx_proto_print("Fake factors = ", temp)

        tx_proto_print("TMB sends (y,q) = (",y,q,") to Sender, with exchange for 1 coin.")
        sender_channel.u2t -= 1
        sender_channel.t2u += 1
        sender.channel_balance -= 1
        tx_proto_print("User with ID "+str(sender.user_id)+ " sent 1 coin to tumbler successfully")



        tx_proto_print("\n\n\n")
        tx_proto_print("--------------[Sender ---> Receiver Communication]-------------------")
        tx_proto_print("Sender sending epsilonStar to Receiver")
        tx_proto_print("Receiver calculating epsilon from epsilonStar")

        epsilonBob=epsilonStar / bf
        tx_proto_print("calculated epsilon at Receiver = ",epsilonBob)
        tx_proto_print("finding sigma from epsilon at Receiver")

        sigmaBob = c - epsilonBob
        tx_proto_print("sigma at Receiver = ",sigmaBob)



        tx_proto_print("\n\n\n")
        tx_proto_print("--------------[Receiver ---> TMB Communication]---------------------")
        tx_proto_print("Puzzle promise protocol exchange")
        for x in range(1,5):
            bf = random.randrange(1000, 5000)
            tx_proto_print("Blinding factor bf = ", bf," Hash = ",hash(bf))
        for x in range(1,5):
            temp = random.randrange(1000, 5000)
            tx_proto_print("Fake factors = ", temp)
        tx_proto_print("Receiver sending sigma = ",sigmaBob," to get 1 coin")

        receiver_channel.u2t += 1
        receiver_channel.t2u -= 1
        receiver.channel_balance += 1

        tx_proto_print("User with ID "+str(receiver.user_id)+ " recived 1 coin successfully")
        amount-=1

    
    print("\nAfter TX execution:\nInformation about Sender and the channel between Sender and Tumbler:")
    print(vars(sender))
    print(vars(sender_channel))
    print("Information about Receiver and the channel between Tumbler and Receiver:")
    print(vars(receiver))
    print(vars(receiver_channel))

In [9]:
number_of_users=27
number_of_probers = 8

tumbler = Tumbler(balance=5000)

users = [User(user_id=i, balance=random.randint(50, 200)) for i in range(1, number_of_users+1)]

probers = [User(user_id=i, balance=random.randint(100, 400)) for i in range(number_of_users+1, number_of_users+number_of_probers+1)]

# attacker =  User(user_id=number_of_users+1, balance=300)

users.extend(probers)


print("\nUsers:")
for i in users:
    print(vars(i))
    
print("\nTumbler Info:")
print(vars(tumbler))

# Users deposit funds
for user in users:
    if user.user_id<=number_of_users:
        desposit_amount=random.randint(10, 30)
        tumbler.create_channel(user, desposit_amount)
    else:
        desposit_amount = int(0.8*user.balance)
        tumbler.create_channel(user, desposit_amount)
        

print("\nUsers (after channel formation):")
for i in users:
    print(vars(i))

    
print("\nTumbler Info (after channel formation):")
print(vars(tumbler))

print("\nChannels with the Tumbler:")
for i in tumbler.channels:
    print(vars(i))

    
num_of_txs = 2

while(num_of_txs):
    
    print("\n\n------------------")
    
    random_selection = random.sample(users, 2)
    
    r_sender = random_selection[0]
    r_receiver = random_selection[1]
    
    r_amount = random.randint(1, 10)

#     print("\n\n",random_selection)
    
#     for i in random_selection:
#         print(vars(i))

    
    transction_protocol(sender=r_sender, receiver=r_receiver, tumbler=tumbler, amount = r_amount)
    num_of_txs-=1





Users:
{'user_id': 1, 'balance': 92, 'channel_balance': 0}
{'user_id': 2, 'balance': 75, 'channel_balance': 0}
{'user_id': 3, 'balance': 174, 'channel_balance': 0}
{'user_id': 4, 'balance': 135, 'channel_balance': 0}
{'user_id': 5, 'balance': 115, 'channel_balance': 0}
{'user_id': 6, 'balance': 160, 'channel_balance': 0}
{'user_id': 7, 'balance': 186, 'channel_balance': 0}
{'user_id': 8, 'balance': 186, 'channel_balance': 0}
{'user_id': 9, 'balance': 144, 'channel_balance': 0}
{'user_id': 10, 'balance': 62, 'channel_balance': 0}
{'user_id': 11, 'balance': 112, 'channel_balance': 0}
{'user_id': 12, 'balance': 91, 'channel_balance': 0}
{'user_id': 13, 'balance': 92, 'channel_balance': 0}
{'user_id': 14, 'balance': 83, 'channel_balance': 0}
{'user_id': 15, 'balance': 162, 'channel_balance': 0}
{'user_id': 16, 'balance': 139, 'channel_balance': 0}
{'user_id': 17, 'balance': 100, 'channel_balance': 0}
{'user_id': 18, 'balance': 69, 'channel_balance': 0}
{'user_id': 19, 'balance': 96, 'chan

In [32]:
def collective_balance_probing(pointer, probers, victim, tumbler):
    
    print("\n\n---------\nVictim Info:",vars(victim))
    
    attacker = probers[pointer]
    amount = attacker.channel_balance

    count=0
    low=0
    high = 0
    
    while(True):

        print("\n--Attacker Info:",vars(attacker))
        print("Pretend sending ",amount)
        fake_TX = Transaction(sender=attacker, receiver=victim, amount=amount, consent=False)
        count+=1
        if amount==(victim.channel_balance):
            print("\nDiscovered Amount:",amount)
            break
        elif amount<(victim.channel_balance):
            low=amount
            print("Error: \"Fake TX!\" from Alice")
        else:
            high=amount
            print("Error: \"Not enough fund!\" from Tumbler")
        amount=int((low+high)/2)
        while(True):
            if pointer<(len(probers)-1):
                pointer+=1
            else:
                pointer=0
                
            if probers[pointer].channel_balance>=amount:
                attacker = probers[pointer]
                break
        

    print("# of probing TX performed:",count,"\n---------\n")
    return pointer

In [33]:
pointer = 0

# collective_balance_probing(pointer, probers, users[0], tumbler)

for victim in users:
    if victim.user_id<=number_of_users:
        pointer = collective_balance_probing(pointer, probers, victim, tumbler)



---------
Victim Info: {'user_id': 1, 'balance': 70, 'channel_balance': 22}

--Attacker Info: {'user_id': 28, 'balance': 24, 'channel_balance': 95}
Pretend sending  95
Error: "Not enough fund!" from Tumbler

--Attacker Info: {'user_id': 29, 'balance': 43, 'channel_balance': 170}
Pretend sending  47
Error: "Not enough fund!" from Tumbler

--Attacker Info: {'user_id': 30, 'balance': 24, 'channel_balance': 92}
Pretend sending  23
Error: "Not enough fund!" from Tumbler

--Attacker Info: {'user_id': 31, 'balance': 45, 'channel_balance': 175}
Pretend sending  11
Error: "Fake TX!" from Alice

--Attacker Info: {'user_id': 32, 'balance': 57, 'channel_balance': 226}
Pretend sending  17
Error: "Fake TX!" from Alice

--Attacker Info: {'user_id': 33, 'balance': 52, 'channel_balance': 206}
Pretend sending  20
Error: "Fake TX!" from Alice

--Attacker Info: {'user_id': 34, 'balance': 23, 'channel_balance': 88}
Pretend sending  21
Error: "Fake TX!" from Alice

--Attacker Info: {'user_id': 35, 'balanc